<a href="https://colab.research.google.com/github/LeeSeungwon89/Kaggle_Dacon_Practice/blob/main/2.%20Porto_Seguro%E2%80%99s_Safe_Driver_Prediction_baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [2]:
ls -1ha kaggle.json

kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

# 참가한 대회 리스트를 확인합니다.
# !kaggle competitions list

In [4]:
!kaggle competitions download -c porto-seguro-safe-driver-prediction

 86% 66.0M/76.5M [00:00<00:00, 238MB/s]
100% 76.5M/76.5M [00:00<00:00, 228MB/s]


In [5]:
!ls

kaggle.json  porto-seguro-safe-driver-prediction.zip  sample_data


In [6]:
!unzip porto-seguro-safe-driver-prediction.zip

Archive:  porto-seguro-safe-driver-prediction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# **1. 피처 엔지니어링**

베이스라인 모델링에서는 반드시 제거해야 할 피처만 제거하고 제거 여부를 확정할 수 없는 피처는 그대로 사용하겠습니다. 성능 개선 챕터에서 여러 방법을 적용해 보겠습니다.

먼저 EDA 수행 결과를 다시 서술하겠습니다.

- 명목형 피처

  - 'ps_ind_02_cat', 'ps_ind_04_cat', 'ps_car_01_cat', 'ps_car_10_cat': 피처를 제거하거나, 결측치인 -1을 고윳값으로 간주하여 예측에 사용할 수 있습니다.

  - 'ps_car_10_cat': 피처를 제거하거나 그대로 사용하는 방안 모두를 적용할 필요가 있습니다.

  - 모든 명목형 피처에 원-핫 인코딩을 적용해야 합니다.

- 이진형 피처

  - 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_13_bin': 신뢰구간이 넓으므로 피처를 제거하거나 예측에 사용하는 방안 모두를 고려해야 합니다.

  - 'ps_calc_15 ~ 20_bin': 고윳값마다 비율 차이가 거의 없으므로 피처를 제거해야 합니다.

- 연속형 피처

  - 'ps_car_01 ~ 03': 구간별 차이가 거의 존재하지 않으므로 피처를 제거해야 합니다.


- 순서형 피처

  - 제거해야 할 피처는 아래와 같습니다.

    - 'ps_calc_04' : 고윳값 비율이 비슷합니다.

  - 제거하거나 사용할 피처는 아래와 같습니다.

    - 'ps_ind_14': 고윳값 비율이 다르지만 고윳값 4의 신뢰구간은 매우 넓습니다. 통계적 유의성이 부족해 보입니다. 다만 신뢰상한점이 높으므로 제거 여부를 확정하기는 어렵습니다.

    - 'ps_calc_05': 고윳값 6의 비율이 높지만 신뢰구간은 넓습니다. 통계적으로 유의하지 않습니다. 다만 신뢰상한점이 높으므로 타깃값 예측력이 존재할 가능성을 배제할 순 없습니다.

    - 'ps_calc_06': 고윳값 비율이 다르지만 고윳값 2와 3의 신뢰구간은 넓습니다. 통계적 유의성이 떨어집니다. 물론 신뢰상한점이 높기 때문에 타깃값 예측력이 없다고 단정하긴 어렵습니다.

    - 'ps_calc_07': 고윳값 비율이 비슷하고 고윳값 9의 신뢰구간은 매우 넓습니다. 통계적 유의성이 떨어집니다. 물론 신뢰상한점이 높기에 타깃값 예측력이 없다고 단정할 수 없습니다.

    - 'ps_calc_08': 여러 고윳값의 비율이 비슷하지만 고윳값 3과 4의 신뢰구간은 넓습니다. 신뢰상한점이 높으므로 타깃값 예측력이 없다고 단정할 수 없습니다.

    - 'ps_calc_09': 고윳값 비율이 비슷하고 고윳값 7의 신뢰구간은 넓습니다. 역시 타깃값 예측력이 부족하다고 단정할 수 없습니다.

    - 'ps_calc_10 ~ 14': 고윳값 비율이 다르지만 신뢰구간은 넓습니다. 통계적 유의성은 부족해 보이지만 타깃값 예측력이 떨어진다고 판단할 수 없습니다. 

  - 모든 순서형 피처에 min-max 정규화를 적용해야 합니다.


## **1.1. 데이터 준비**

데이터를 준비하겠습니다.

In [7]:
import numpy as np
import pandas as pd

# 최대 행렬 수를 설정합니다.
pd.set_option('display.max_column', 80)
pd.set_option('display.max_rows', 80)

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [8]:
all_data = pd.concat([train, test], ignore_index=True) # 인덱스를 순차형으로 지정합니다.
all_data

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0.0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0.0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0.0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0.0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0.0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488023,1488022,NaN,0,1,6,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0.5,0.3,1.048809,0,1,-1,8,0,14,0,1,0,1,18,2,0.565685,1.528774,0.565685,3.605551,0.3,0.4,0.9,1,1,7,3,10,3,10,4,2,3,4,0,1,0,0,1,0
1488024,1488023,NaN,5,3,5,1,0,0,0,1,0,0,0,0,0,0,11,1,0,0,0.7,1.0,1.246495,8,1,-1,0,0,10,1,0,0,1,83,2,0.424264,1.040567,0.418569,3.741657,0.3,0.2,0.6,1,3,7,3,10,4,9,6,2,2,11,0,0,1,1,0,0
1488025,1488024,NaN,0,1,5,0,0,1,0,0,0,0,0,0,0,0,5,0,0,1,0.4,0.0,0.609303,11,1,-1,0,-1,11,1,0,1,1,17,2,0.374166,0.968992,0.390384,3.741657,0.3,0.3,0.9,2,1,10,4,8,2,8,5,2,2,11,0,1,1,0,0,0
1488026,1488025,NaN,6,1,5,1,0,0,0,0,1,0,0,0,0,0,13,1,0,0,0.6,0.6,0.920937,10,0,-1,0,1,10,1,1,0,1,28,3,0.387298,0.725125,0.378153,2.645751,0.1,0.1,0.3,1,1,9,2,10,1,8,1,1,2,7,1,1,0,0,0,0


In [9]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1488028 entries, 0 to 1488027
Data columns (total 59 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   id              1488028 non-null  int64  
 1   target          595212 non-null   float64
 2   ps_ind_01       1488028 non-null  int64  
 3   ps_ind_02_cat   1488028 non-null  int64  
 4   ps_ind_03       1488028 non-null  int64  
 5   ps_ind_04_cat   1488028 non-null  int64  
 6   ps_ind_05_cat   1488028 non-null  int64  
 7   ps_ind_06_bin   1488028 non-null  int64  
 8   ps_ind_07_bin   1488028 non-null  int64  
 9   ps_ind_08_bin   1488028 non-null  int64  
 10  ps_ind_09_bin   1488028 non-null  int64  
 11  ps_ind_10_bin   1488028 non-null  int64  
 12  ps_ind_11_bin   1488028 non-null  int64  
 13  ps_ind_12_bin   1488028 non-null  int64  
 14  ps_ind_13_bin   1488028 non-null  int64  
 15  ps_ind_14       1488028 non-null  int64  
 16  ps_ind_15       1488028 non-null  in

'id'와 'target' 피처를 제거하겠습니다.

In [10]:
all_data.drop(['id', 'target'], axis=1, inplace=True)

## **1.2. 불필요한 피처 제거**

반드시 제거해야 할 피처를 모두 제거하겠습니다.

In [11]:
drop_all = ['ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
            'ps_calc_19_bin', 'ps_calc_20_bin', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03',
            'ps_calc_04']
all_data.drop(drop_all, axis=1, inplace=True)

In [12]:
all_data.head(3)

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14
0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,1,10,1,10,1,5,9,1,5,8
1,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,1,9,5,8,1,7,3,1,1,9
2,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,2,9,1,8,2,7,4,2,7,7


## **1.3. 명목형 피처에 원-핫 인코딩**

명목형 피처에 원-핫 인코딩을 수행하겠습니다. 먼저 각 피처명을 리스트에 분류하겠습니다.

In [13]:
list_category =[]
list_bin = []
list_continuous = []
list_ordinal = []

for column in all_data.columns:
    if 'cat' in column:
        list_category.append(column)
    elif ('bin' in column):
        list_bin.append(column)
    elif all_data[column].dtype == 'float64':
        list_continuous.append(column)
    elif (all_data[column].dtype == 'int64'):
        list_ordinal.append(column)

명목형 피처에 원-핫 인코딩을 수행하여 따로 객체화하겠습니다. 아래에서 전체 데이터와 결합할 예정입니다.

In [14]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()
category_one_hot_encoded = one_hot_encoder.fit_transform(all_data[list_category])

In [15]:
category_one_hot_encoded

<1488028x184 sparse matrix of type '<class 'numpy.float64'>'
	with 20832392 stored elements in Compressed Sparse Row format>

전체 데이터에서 명목형 피처를 제거해 두겠습니다.

In [16]:
all_data.drop(list_category, axis=1, inplace=True)

## **1.4. 순서형 피처에 min-max 정규화**

순서형 피처에 min-max 정규화를 적용하겠습니다.

In [17]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
all_data[list_ordinal] = min_max_scaler.fit_transform(all_data[list_ordinal])

In [18]:
all_data[list_ordinal].head(3)

,ps_ind_01,ps_ind_03,ps_ind_14,ps_ind_15,ps_car_11,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14
0,0.285714,0.454545,0.0,0.846154,0.75,0.166667,1.0,0.111111,0.818182,0.142857,0.20,0.45,0.090909,0.333333,0.285714
1,0.142857,0.636364,0.0,0.230769,1.00,0.166667,0.9,0.555556,0.636364,0.142857,0.28,0.15,0.090909,0.066667,0.321429
2,0.714286,0.818182,0.0,0.923077,0.50,0.333333,0.9,0.111111,0.636364,0.285714,0.28,0.20,0.181818,0.466667,0.250000


## **1.5. 전체 데이터와 매트릭스 결합**

전체 데이터와 원-핫 인코딩 된 명목형 피처 매트릭스를 결합하겠습니다. 

In [19]:
from scipy import sparse

all_data_sparse_hstack = sparse.hstack([sparse.csr_matrix(all_data),
                                       category_one_hot_encoded], format='csr')

In [20]:
all_data_sparse_hstack

<1488028x217 sparse matrix of type '<class 'numpy.float64'>'
	with 53373507 stored elements in Compressed Sparse Row format>

# **2. 모델링**

## **2.1. 데이터 준비**

데이터를 준비하겠습니다.

In [21]:
size_train = len(train)
X_train_sparse = all_data_sparse_hstack[:size_train]
X_test_sparse = all_data_sparse_hstack[size_train:]
y_train_original = train['target'].values

## **2.2. 정규화된 지니계수 산출 함수 선언**

정규화된 지니계수를 계산하는 두 가지 동일한 함수를 선언하겠습니다. 참고로 지니계수는 값이 클수록 불평등함을 의미하고, 정규화된 지니계수는 값이 클수록 좋은 성능을 의미합니다. 선언한 두 함수 중 후자의 함수를 사용하겠습니다. 

In [22]:
# def normalized_gini_coefficient_1(y_true, y_prediction):
#     # `y_true`: 실젯값이 들어있는 배열
#     # `y_prediction`: 예측 값이 들어있는 배열
#     # 예측 값은 오름차순으로 정렬되어야 합니다.

#     n = len(y_true) # 실젯값의 길이를 객체화합니다.

#     y_true_sum = np.sum(y_true) # 실젯값을 모두 합산합니다.

#     # `y_true`와 `y_prediction`의 차이의 합을 구합니다.
#     # 이 합을 이용하여 최대 불평등도를 계산할 수 있습니다.
#     difference_sum = np.sum(np.abs(y_true - y_prediction))

#     # 최대 불평등도를 구합니다.
#     max_inequality = np.sum(y_true * (np.arange(1, n + 1) / n)) - (y_true_sum / n)

#     # 정규화된 지니계수를 계산하여 반환합니다. 최대 불평등도가 1이면 0을 반환합니다.
#     if max_inequality == 0:
#         return 1.0
#     else:
#         return (max_inequality - difference_sum / n) / max_inequality

In [23]:
def normalized_gini_coefficient_2(y_true, y_prediction): # `y_true`는 실젯값, `y_prediction`은 예측값입니다.
    # 실젯값 크기와 예측값 크기를 대조합니다.
    assert y_true.shape == y_prediction.shape

    number_samples = y_true.shape[0] # 데이터 개수를 객체화합니다.
    lolenz_middle = np.linspace(1 / number_samples, 1, number_samples) # 대각선 값을 객체화합니다.

    # 예측값의 지니계수를 산출합니다.
    prediction_order = y_true[y_prediction.argsort()] # `y_prediction` 크기 순서에 맞추어 `y_true` 값을 재정렬합니다.
    lolenz_prediction = np.cumsum(prediction_order) / np.sum(prediction_order) # 로렌츠 곡선을 객체화합니다.
    gini_prediction = np.sum(lolenz_middle - lolenz_prediction) 

    # 모든 예측이 정확한 경우의 지니계수를 산출합니다.
    true_order = y_true[y_true.argsort()] # `y_true` 크기 순서에 맞추어 `y_true` 값을 재정렬합니다.
    lolenz_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선을 객체화합니다.
    gini_true = np.sum(lolenz_middle - lolenz_true)

    # 정규화된 지니계수를 산출합니다.
    normalized_gini_coefficient = gini_prediction / gini_true

    return normalized_gini_coefficient

## **2.3. 층화 K-폴드 및 OOF 예측을 위한 변수 준비**

타깃값이 불균형하므로 층화 K 폴드 방식을 채택하겠습니다. 아울러 성능 측정에 OOF prediction 방식을 활용하기 위해 예측 확률을 저장하는 1차원 배열 객체를 생성하겠습니다.

In [24]:
from sklearn.model_selection import StratifiedKFold

stratified_kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)

# 1차원 배열을 생성합니다.
oof_valid_predictions = np.zeros(X_train_sparse.shape[0])
oof_test_predictions = np.zeros(X_test_sparse.shape[0])

## **2.4. 모델 훈련**

데이터셋 크기가 크기 때문에 훈련 속도가 빠른 LightGBM을 적용하겠습니다. 이번에는 파이썬 래퍼 형식으로 훈련해 보겠습니다.

In [25]:
# 연습 코드(사이킷런 래퍼 형식)
# from lightgbm import LGBMClassifier
# from sklearn.metrics import roc_curve
# from sklearn.metrics import make_scorer

# parameters = {
#     'learning_rate': 0.01,
#     'n_estimators': 5000,
#     'objective': 'binary'
#     }

# for index, (index_train, index_valid) in enumerate(stratified_kfolds.split(X_train_sparse, y_train_original)):
#     print('###', f'fold {index + 1} - fold {stratified_kfolds.n_splits}', '###')

#     X_train, y_train = X_train_sparse[index_train], y_train_original[index_train]
#     X_valid, y_valid = X_train_sparse[index_valid], y_train_original[index_valid]

#     lgb_model = LGBMClassifier(**parameters, n_jobs=-1)

#     lgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=eval_gini, 
#                   early_stopping_rounds=100, verbose=100)
    
#     # 테스트 데이터를 활용해 OOF 예측
#     oof_test_prediction += lgb_model.predict_proba(X_test)[:, 1] / stratified_kfolds.n_splits
    
#     # 모델 성능 평가를 위한 검증 데이터 타깃값 예측
#     oof_valid_prediction[index_valid] += lgb_model.predict_proba(X_valid)[:, 1]
    
#     # 검증 데이터 예측 확률에 대한 정규화 지니계수 
#     gini_score = normalized_gini_coefficient_2(y_valid, oof_valid_prediction[index_valid])
#     print(f'folds: {index + 1} - gini score: {gini_score}\n')

In [26]:
# 사이킷런 래퍼 형식
# import numpy as np
# from lightgbm import LGBMClassifier
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import make_scorer

# def eval_gini(y_true, y_pred):
#     assert y_true.shape == y_pred.shape

#     n_samples = y_true.shape[0]                      
#     L_mid = np.linspace(1 / n_samples, 1, n_samples) 

#     pred_order = y_true[y_pred.argsort()] 
#     L_pred = np.cumsum(pred_order) / np.sum(pred_order)
#     G_pred = np.sum(L_mid - L_pred)

#     true_order = y_true[y_true.argsort()] 
#     L_true = np.cumsum(true_order) / np.sum(true_order)
#     G_true = np.sum(L_mid - L_true)

#     return G_pred / G_true

# gini_scorer = make_scorer(eval_gini, greater_is_better=True, needs_proba=True)

# folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

# params = {'objective': 'binary',
#           'learning_rate': 0.01,
#           'force_row_wise': True,
#           'random_state': 0}

# oof_val_preds = np.zeros(X_train_sparse.shape[0]) 
# oof_test_preds = np.zeros(X_test_sparse.shape[0]) 

# for idx, (train_idx, valid_idx) in enumerate(folds.split(X_train_sparse, y_train_original)):
#     print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
#     X_train, y_train = X_train_sparse[train_idx], y_train_original[train_idx] 
#     X_valid, y_valid = X_train_sparse[valid_idx], y_train_original[valid_idx] 

#     lgb_model = LGBMClassifier(**params)
#     lgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], 
#                   eval_metric=gini_scorer, early_stopping_rounds=100, verbose=100)
    
#     oof_test_preds += lgb_model.predict(X_test_sparse) / folds.n_splits
#     oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
#     gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
#     ngini_score = gini_score / eval_gini(y_valid, y_valid)
#     print(f'폴드 {idx+1} 지니계수 : {gini_score}, 정규화 지니계수 : {ngini_score}\n')

In [27]:
# 파이썬 래퍼 형식
import lightgbm as lgb

# 모델을 훈련할 때 검증용으로 지정할 함수를 선언합니다.
def gini_feval(predictions, dtrain):
    labels = dtrain.get_label() # 데이터셋의 타깃값을 반환합니다.
    return 'gini', normalized_gini_coefficient_2(labels, predictions), True
# 함수 끝

parameters = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'force_row_wise': True, # 경고 메시지를 띄우지 않습니다.
    'random_state': 2023
}

for index, (train_index, valid_index) in enumerate(stratified_kfolds.split(X_train_sparse, y_train_original)):
    print('---', f'fold {index + 1} in folds {stratified_kfolds.n_splits}', '---')

    X_train, y_train = X_train_sparse[train_index], y_train_original[train_index]
    X_valid, y_valid = X_train_sparse[valid_index], y_train_original[valid_index]

    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_valid, y_valid)

    lgb_model = lgb.train(params=parameters,
                          train_set=dtrain,
                          num_boost_round=5000, # 부스팅 반복 횟수를 지정합니다.
                          valid_sets=dvalid, 
                          feval=gini_feval, # 검증 평가 지표로 `gini_feval` 함수를 지정합니다.
                          early_stopping_rounds=100, # 100회 안에 성능이 향상되지 않으면 종료합니다.
                          verbose_eval=50) # 50번째에서 점수를 출력합니다.
    
    # 테스트 세트로 OOF 예측을 수행합니다.
    oof_test_predictions += lgb_model.predict(X_test_sparse)/stratified_kfolds.n_splits
    # 검증 세트로 타깃값을 예측하여 모델 성능을 측정합니다.
    oof_valid_predictions[valid_index] += lgb_model.predict(X_valid)
    
    # 검증 세트로 타깃값을 예측한 배열을 토대로 정규화된 지니계수를 산출합니다.
    normalized_gini_coefficient_final = normalized_gini_coefficient_2(y_valid, oof_valid_predictions[valid_index])
    print(f'fold {index+1}\'s normalized gini coefficient: {normalized_gini_coefficient_final}\n')

--- fold 1 in folds 5 ---
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.154288	valid_0's gini: 0.245601
[100]	valid_0's binary_logloss: 0.153297	valid_0's gini: 0.258014
[150]	valid_0's binary_logloss: 0.152767	valid_0's gini: 0.264622
[200]	valid_0's binary_logloss: 0.152435	valid_0's gini: 0.26941
[250]	valid_0's binary_logloss: 0.152237	valid_0's gini: 0.272142
[300]	valid_0's binary_logloss: 0.152113	valid_0's gini: 0.273908
[350]	valid_0's binary_logloss: 0.152034	valid_0's gini: 0.27525
[400]	valid_0's binary_logloss: 0.151984	valid_0's gini: 0.275887
[450]	valid_0's binary_logloss: 0.151924	valid_0's gini: 0.277201
[500]	valid_0's binary_logloss: 0.151899	valid_0's gini: 0.277522
[550]	valid_0's binary_logloss: 0.151873	valid_0's gini: 0.278243
[600]	valid_0's binary_logloss: 0.151868	valid_0's gini: 0.278499
[650]	valid_0's binary_logloss: 0.151855	valid_0's gini: 0.278877
[700]	valid_0's binary_logloss: 0.15186	valid_0's gini: 

## **2.5. 성능 측정 및 파일 제출**

In [28]:
normalized_gini_coefficient_2(y_train_original, oof_valid_predictions)

0.2792280251286247

In [29]:
submission['target'] = oof_test_predictions
submission.to_csv('Porto Seguro’s Safe Driver Prediction_submission_2.csv', index=False)

프라이빗 스코어는 0.28266입니다. 5156팀 중 2731위에 위치합니다. 성능 향상을 도모하기 위해 피처를 추가로 제거하고 하이퍼파라미터를 튜닝할 필요가 있습니다. 추가로 파생 피처를 생성해 볼 수도 있습니다.